<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
# default_exp trainer.hifi_gan

In [ ]:
# export
import itertools

import torch
from torch.nn.parallel import DistributedDataParallel
from torch.optim import AdamW
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import DistributedSampler, DataLoader

from uberduck_ml_dev.data_loader import AudioMelDataset
from uberduck_ml_dev.models.hifi_gan import (
    Generator,
    MultiPeriodDiscriminator,
    MultiScaleDiscriminator,
)
from uberduck_ml_dev.trainer.base import TTSTrainer


class HifiGANTrainer(TTSTrainer):
    @property
    def training_dataset_args(self):
        return [
            self.training_audiopaths_and_text,
            self.segment_size,
            self.n_mel_channels,
            self.sample_rate,
            self.mel_fmin,
            self.mel_fmax,
            self.mel_fmax_for_loss,
            self.filter_length,
            self.hop_length,
            self.win_length,
        ]

    @property
    def val_dataset_args(self):
        val_args = [a for a in self.training_dataset_args]
        val_args[0] = self.val_audiopaths_and_text
        return val_args

    def train(self):

        generator = Generator(self.hparams)
        mpd = MultiPeriodDiscriminator()
        msd = MultiScaleDiscriminator()

        optim_g = AdamW(
            generator.parameters(),
            lr=self.learning_rate,
            betas=[self.adam_b1, self.adam_b2],
        )
        optim_d = AdamW(
            itertools.chain(mpd.parameters(), msd.parameters()),
            lr=self.learning_rate,
            betas=[self.adam_b1, self.adam_b2],
        )
        scheduler_g = ExponentialLR(
            optim_g, gamma=self.lr_decay, last_epoch=self.epochs
        )
        scheduler_d = ExponentialLR(
            optim_d, gamma=self.lr_decay, last_epoch=self.epochs
        )

        generator.train()
        mpd.train()
        msd.train()
        # main training loop
        for epoch in range(start_epoch, self.epochs):
            pass

In [ ]:
AudioMelDataset??